Evaluation
============

We use the terms from [arc42](https://docs.arc42.org/section-7/) for the different views.

Building Block View Trajectory Generation and Evaluation
--------------------------------------------------------

This is a conceptual view and reflects the target state of the implementation. In Flatland implementation, we currently do not yet distinguish between configuration and state, they go together in `RailEnvPersister` and a trajectory currently consists of full snapshots.

```mermaid
classDiagram
    class Runner {
        Trajectory: +generate_trajectory_from_policy(Policy policy, ObservationBuilder obs_builder, int snapshot_interval)$ Trajectory
    }
    
    class Evaluator {
        Trajectory: +evaluate(Trajectory trajectory)
    }
    
    class Trajectory {
        Trajectory: +Path data_dir
        Trajectory: +UUID ep_id
        Trajectory: +run(int from_step, int to_step=-1)
    }



    

    class EnvSnapshot {
        EnvSnapshot: +Path data_dir
        Trajectory: +UUID ep_id
    }

    class EnvConfiguration
    EnvConfiguration: +int max_episode_steps
    EnvConfiguration: +int height
    EnvConfiguration: +int width
    EnvConfiguration: +Rewards reward_function
    EnvConfiguration: +MalGen
    EnvConfiguration: +RailGen etc. reset

    class EnvState {
        EnvState: +Grid rail
    }

    
        class EnvConfiguration

        class EnvState

        class EnvSnapshot
        
        class EnvActions

        class EnvRewards
    

    

    EnvSnapshot --> "1" EnvConfiguration
    EnvSnapshot --> "1" EnvState
    Trajectory --> "1" EnvConfiguration
    Trajectory --> "1..*" EnvState
    Trajectory --> "1..*" EnvActions
    Trajectory --> "1..*" EnvRewards

    class Policy
    Policy: act(int handle, Observation observation)

    class ObservationBuilder
    ObservationBuilder: get()
    ObservationBuilder: get_many()

    class Submission
    Submission --> "1" Policy
    Submission --> ObservationBuilder
```

Remarks:

* Trajectory needs not start at step 0
* Trajectory needs not contain state for every step - however, when starting the trajectory from an intermediate step, the snapshot must exist.

Runtime View Trajectory Generation
--------------------------------

```mermaid
flowchart TD
    subgraph Trajectory.create_from_policy
        start(("&nbsp;")) -->|data_dir| D0
        D0(RailEnvPersister.load_new) -->|env| E{env done?}
        E -->|no:<br/>observations| G{Agent loop:<br/> more agents?}
        G --->|observation| G1(policy.act)
        G1 -->|action| G
        G -->|no:<br/> actions| F3(env.step)
        F3 -->|observations,rewards,info| E
        E -->|yes:<br/> rewards| H((("&nbsp;")))
    end

    style Policy fill: #ffe, stroke: #333, stroke-width: 1px, color: black
    style G1 fill: #ffe, stroke: #333, stroke-width: 1px, color: black
    style Env fill: #fcc, stroke: #333, stroke-width: 1px, color: black
    style F3 fill: #fcc, stroke: #333, stroke-width: 1px, color: black
    subgraph legend
        Env(Environment)
        Policy(Policy)
        Trajectory(Trajectory)
    end

    Trajectory.create_from_policy~~~legend
```

Trajectory Generation and Evaluation
------------------------------------

Create a trajectory from a random policy and inspect the output

In [13]:
import tempfile
from pathlib import Path
from typing import Any, Optional

from flatland.env_generation.env_generator import env_generator
from flatland.trajectories.trajectories import Policy
from flatland.utils.seeding import np_random, random_state_to_hashablestate
from flatland.evaluators.trajectory_evaluator import TrajectoryEvaluator, evaluate_trajectory
from flatland.trajectories.trajectories import Trajectory

In [4]:
class RandomPolicy(Policy):
    def __init__(self, action_size: int = 5, seed=42):
        super(RandomPolicy, self).__init__()
        self.action_size = action_size
        self.np_random, _ = np_random(seed=seed)

    def act(self, handle: int, observation: Any, **kwargs):
        return self.np_random.choice(self.action_size)

In [25]:
with tempfile.TemporaryDirectory() as tmpdirname:
    data_dir = Path(tmpdirname)
    trajectory = Trajectory.create_from_policy(policy=RandomPolicy(), data_dir=data_dir, snapshot_interval=15)
    # np_random in loaded episode is same as if it comes directly from env_generator incl. reset()!
    env = trajectory.restore_episode()
    gen, _, _ = env_generator()
    assert random_state_to_hashablestate(env.np_random) == random_state_to_hashablestate(gen.np_random)

    
    # inspect output
    for p in sorted(data_dir.rglob("**/*")):
        print(p)

    # inspect the actions taken by the policy
    print(trajectory.read_actions())

    # verify steps 5 to 15 - we can start at 5 as there is a snapshot for step 5.
    TrajectoryEvaluator(trajectory).evaluate(start_step=15,end_step=25)

/Users/che/Miniconda3/miniconda3/envs/flatland-book/lib/python3.12/site-packages/flatland/envs/rail_generators.py:321: UserWarning: Could not set all required cities! Created 1/2
  warnings.warn(city_warning)
/Users/che/Miniconda3/miniconda3/envs/flatland-book/lib/python3.12/site-packages/flatland/envs/rail_generators.py:217: UserWarning: [WARNING] Changing to Grid mode to place at least 2 cities.
  warnings.warn("[WARNING] Changing to Grid mode to place at least 2 cities.")
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 470/471 [00:07<00:00, 66.92it/s]
/Users/che/Miniconda3/miniconda3/envs/flatland-book/lib/python3.12/site-packages/flatland/envs/rail_generators.py:321: UserWarning: Could not set all requir

/var/folders/4x/y75rw_lj5xx6xc0bmv1gxmq80000gn/T/tmpzhwsrjjg/event_logs
/var/folders/4x/y75rw_lj5xx6xc0bmv1gxmq80000gn/T/tmpzhwsrjjg/event_logs/ActionEvents.discrete_action.tsv
/var/folders/4x/y75rw_lj5xx6xc0bmv1gxmq80000gn/T/tmpzhwsrjjg/event_logs/TrainMovementEvents.trains_arrived.tsv
/var/folders/4x/y75rw_lj5xx6xc0bmv1gxmq80000gn/T/tmpzhwsrjjg/event_logs/TrainMovementEvents.trains_positions.tsv
/var/folders/4x/y75rw_lj5xx6xc0bmv1gxmq80000gn/T/tmpzhwsrjjg/outputs
/var/folders/4x/y75rw_lj5xx6xc0bmv1gxmq80000gn/T/tmpzhwsrjjg/serialised_state
/var/folders/4x/y75rw_lj5xx6xc0bmv1gxmq80000gn/T/tmpzhwsrjjg/serialised_state/4eb42575-e47d-4a5e-b96a-420d0947e174.pkl
/var/folders/4x/y75rw_lj5xx6xc0bmv1gxmq80000gn/T/tmpzhwsrjjg/serialised_state/4eb42575-e47d-4a5e-b96a-420d0947e174_step0000.pkl
/var/folders/4x/y75rw_lj5xx6xc0bmv1gxmq80000gn/T/tmpzhwsrjjg/serialised_state/4eb42575-e47d-4a5e-b96a-420d0947e174_step0015.pkl
/var/folders/4x/y75rw_lj5xx6xc0bmv1gxmq80000gn/T/tmpzhwsrjjg/serialised_state

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 87.16it/s]

0.0% trains arrived. Expected 0.0%. 24 elapsed steps.


Flatland Callbacks
------------------
Flatland callbacks can be used for custom metrics and custom postprocessing.

In [42]:
import inspect
from flatland.envs.rail_env import RailEnv
from flatland.callbacks.callbacks import FlatlandCallbacks, make_multi_callbacks

In [57]:
lines, _ = inspect.getsourcelines(FlatlandCallbacks)
for s in lines:
    print(s)

class FlatlandCallbacks:

    """

    Abstract base class for Flatland callbacks similar to rllib, see https://github.com/ray-project/ray/blob/master/rllib/callbacks/callbacks.py.



    These callbacks can be used for custom metrics and custom postprocessing.



    By default, all of these callbacks are no-ops.

    """



    def on_episode_start(

        self,

        *,

        env: Optional[Environment] = None,

        data_dir: Path = None,

        **kwargs,

    ) -> None:

        """Callback run right after an Episode has been started.



        This method gets called after `env.reset()`.



        Parameters

        ---------

            env : Environment

                the env

            data_dir : Path

                trajectory data dir

            kwargs:

                Forward compatibility placeholder.

        """

        pass



    def on_episode_step(

        self,

        *,

        env: Optional[Environment] = None,

        data_dir: Path 

In [44]:
class DummyCallbacks(FlatlandCallbacks):
    def on_episode_step(
        self,
        *,
        env: Optional[RailEnv] = None,
        **kwargs,
    ) -> None:
        print(f"step{env._elapsed_steps - 1}")

/Users/che/Miniconda3/miniconda3/envs/flatland-book/lib/python3.12/site-packages/flatland/envs/rail_generators.py:321: UserWarning: Could not set all required cities! Created 1/2
  warnings.warn(city_warning)
/Users/che/Miniconda3/miniconda3/envs/flatland-book/lib/python3.12/site-packages/flatland/envs/rail_generators.py:217: UserWarning: [WARNING] Changing to Grid mode to place at least 2 cities.
  warnings.warn("[WARNING] Changing to Grid mode to place at least 2 cities.")
  6%|████████████████████                                                                                                                                                                                                                                                                                                                 | 29/471 [00:00<00:04, 93.10it/s]

step0
step1
step2
step3
step4
step5
step6
step7
step8
step9
step10
step11
step12
step13
step14
step15
step16
step17
step18
step19
step20
step21
step22
step23
step24
step25
step26
step27
step28


 10%|█████████████████████████████████▊                                                                                                                                                                                                                                                                                                   | 49/471 [00:00<00:04, 93.82it/s]

step29
step30
step31
step32
step33
step34
step35
step36
step37
step38
step39
step40
step41
step42
step43
step44
step45
step46
step47
step48
step49
step50
step51
step52
step53
step54
step55
step56
step57
step58


 17%|██████████████████████████████████████████████████████▌                                                                                                                                                                                                                                                                              | 79/471 [00:00<00:04, 92.63it/s]

...

step409
step410
step411
step412
step413
step414
step415
step416
step417
step418
step419
step420
step421
step422
step423
step424
step425
step426
step427
step428
step429
step430
step431
step432
step433
step434
step435
step436
step437
step438


 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋        | 459/471 [00:04<00:00, 94.93it/s]

step439
step440
step441
step442
step443
step444
step445
step446
step447
step448
step449
step450
step451
step452
step453
step454
step455
step456
step457
step458


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 470/471 [00:05<00:00, 93.49it/s]

step459
step460
step461
step462
step463
step464
step465
step466
step467
step468
step469
step470
0.0% trains arrived. Expected 0.0%. 470 elapsed steps.


In [ ]:
with tempfile.TemporaryDirectory() as tmpdirname:
    data_dir = Path(tmpdirname)
    trajectory = Trajectory.create_from_policy(policy=RandomPolicy(), data_dir=data_dir, snapshot_interval=15)
    TrajectoryEvaluator(trajectory, callbacks=make_multi_callbacks(DummyCallbacks())).evaluate()